# 03-01 Fakten Demo

## Hinweise zur Übung

Ziel dieser Übung ist die beispielhafte Erzeugung von Faktentabellen für das Data Warehouse-Szenario. Hierzu nutzen wir die Inhalte der Staging-Datenbanken als Datenquelle und transformieren daraus die Faktendaten und liefern sie in die Reporting-Datenbank aus.

Die Übung setzt auf den Ergebnissen der Extraktions (Übung 01) auf. Da in der Übung nur einzelne Tabellen in die Staging-Datenbanken übertragen wurden, setzen wir hier mit einem einheitlichen Stand aus vorbereiteten SQLite-Datenbanken auf, in die alle TSV-Dateien bereits importiert wurden.

Die Tabellenstruktur der zu erstellenden Faktentabellen inkl. der Spalten (nicht jedoch Datentypen) geht aus dem ER-Diagramm im `data`-Verzeichnis hervor. Die Datentypen wiederum können in den Dokumentationen der jeweiligen Qeulldatenbereiche (Bielefeld, Mannheim, Dimensionen) nachgeschlagen werden.

## Konfiguration des Notebooks

In [ ]:
# Ggf. fehlende Pakete installieren
!pip install --quiet ipython-sql

In [ ]:
import os
import sys
import urllib.request
%load_ext sql

In [ ]:
# Konfiguration
base_url_quellen = "https://raw.githubusercontent.com/fau-lmi/lct-ehealth/main/07-ETL+DWH/data"
base_url_staging = "./"

In [ ]:
# SQlite-Datenbanken aus Github auf den Jupyter-Server herunterladen
urllib.request.urlretrieve(base_url_quellen + "/datenbanken/stg_bielefeld.sqlite",  "stg_bielefeld.sqlite")
urllib.request.urlretrieve(base_url_quellen + "/datenbanken/stg_mannheim.sqlite",   "stg_mannheim.sqlite")
urllib.request.urlretrieve(base_url_quellen + "/datenbanken/stg_dimensions.sqlite", "stg_dimensions.sqlite")
urllib.request.urlretrieve(base_url_quellen + "/datenbanken/staging.sqlite",        "staging.sqlite")

In [ ]:
# Datenbankverbindung als Pfad (für das ETL) & iPython SQL (für die Abfragen) herstellen
db_path_stg_bielefeld  = base_url_staging + "stg_bielefeld.sqlite"
db_path_stg_mannheim   = base_url_staging + "stg_mannheim.sqlite"
db_path_stg_dimensions = base_url_staging + "stg_dimensions.sqlite"
db_path_staging        = base_url_staging + "staging.sqlite"
db_path_reporting      = base_url_staging + "reporting.sqlite"

db_url_stg_bielefeld  = "sqlite:///" + db_path_stg_bielefeld
db_url_stg_mannheim   = "sqlite:///" + db_path_stg_mannheim
db_url_stg_dimensions = "sqlite:///" + db_path_stg_dimensions
db_url_staging        = "sqlite:///" + db_path_staging
db_url_reporting      = "sqlite:///" + db_path_reporting

%sql $db_url_stg_bielefeld
%sql $db_url_stg_mannheim
%sql $db_url_stg_dimensions
%sql $db_url_staging
%sql $db_url_reporting

In [ ]:
%%sql $db_url_staging
ATTACH DATABASE :db_path_stg_bielefeld  AS bielefeld;
ATTACH DATABASE :db_path_stg_mannheim   AS mannheim;
ATTACH DATABASE :db_path_stg_dimensions AS dimensions;
ATTACH DATABASE :db_path_reporting      AS reporting;

In [ ]:
# Dimension D_FALLART erzeugen (wird für die Generierung der Tabelle F_FAELLE benötigt)
%%sql $db_url_staging
DROP TABLE IF EXISTS reporting.d_fallart;

CREATE TABLE reporting.d_fallart (
  fallart_id INTEGER PRIMARY KEY AUTOINCREMENT,
  fallart_name VARCHAR(50)
);

INSERT INTO reporting.d_fallart (fallart_name)

SELECT DISTINCT
       fallart AS fallart_name
  FROM bielefeld.faelle

UNION

SELECT DISTINCT
       encounterclass AS fallart_name
  FROM mannheim.encounters;

## Faktentabelle F_FAELLE

Der Behandlungsfall ist ein zentrales Element der Dokumentation in Krankenhäusern, da in der Regel alle während des Falls festgestellten Diagnosen, durchgeführten Untersuchungen und erbrachten Leistungen mit Bezug auf den Fall erfasst und auch für die Abrechnung genutzt werden.

Behandlungsfälle beziehen sich auf einen Patientenstammsatz und eine Einrichtung, in der der Fall behandelt wurde. Fälle haben eine Fallart (z.B. ambulant/stationär) sowie einen Anfangs- & Endzeitpunkt. Außerdem sind typischerweise eine Hauptdiagnose, die Dauer des Falls sowie ggf. Erlöse dokumentiert.

Im folgenden Block erzeugen wir die Faktentabelle F_FAELLE. Sie stellt den wiederholenden Kern der nachfolgenden Faktentabellen dar, da wir alle Eigenschaften als "Kopf" der weiteren (1:n-zugeordneten) Faktentabellen nachnutzen werden. Die zu generierenden Spalten haben wir im Datenmodell des Kurs-DWH festgelegt.


### 1. Zieltabelle anlegen

Hierzu legen wir zunächst mit einem CREATE-Table die Tabelle in der Reporting-Datenbank (leer) an.

In [ ]:
%%sql $db_url_staging
DROP TABLE IF EXISTS reporting.f_faelle;

CREATE TABLE reporting.f_faelle (
  fall_id                 INTEGER,
  patient_id              INTEGER,
  einrichtung_id          INTEGER,
  aufnahme_datum          VARCHAR(10),
  aufnahme_datumzeit      VARCHAR(18),
  entlass_datum           VARCHAR(10),
  entlass_datumzeit       VARCHAR(18),
  fallart_id              INTEGER,
  aufnahmegrund_id        VARCHAR(20),
  hauptdiagnose_snomed_id VARCHAR(20),
  aufnahmealter_jahre     FLOAT,
  liegedauer_tage         FLOAT,
  erloes_fallpauschale    FLOAT
);

### 2. Daten aus den Quelltabellen auslesen & in Zieltabelle schreiben

Die Rohdaten dieser Faktentabelle stammen fast vollständig aus der Fall- bzw. Encountertabelle der jeweiligen Datenquelle. Zur Berechnung des Alters müssen wir allerdings zusätzlich das Geburtsdatum aus den Patientenstammdaten einbeziehen und ermitteln es durch Subtraktion des Geburts- und Aufnahmedatums). Außerdem müssen wir für die Ermittlung der Fallart-ID die Fallarten-Dimension joinen.

Wir fügen auch hier Daten aus beiden Quellbereichen (Bielefeld & Mannheim) zusammen, um übergreifende Auswertungen zu ermöglichen. Wir nutzen dazu wieder das `UNION`-Statement, um die beiden Teilabfragen auf die Datenquellen hintereinander zu stellen. Da sich die Nummernkreise der Einrichtungen, Patienten und Fälle zwischen beiden Quellen überschneiden, müssen wir die IDs jeweils durch einen Präfix ("B-" bzw. "M-") "qualifizieren", also eindeutig machen.

Das benötigte Statement sieht zunächst lang und komplex aus, aber wie zuvor auch sind beide Teile (für Bielefeld und Mannheim) fast deckungsgleich, es müssen lediglich die Spaltennamen angepasst und die beiden Teilabfragen zusammengeführt werden.

Für die Reihenfolge der Spalten hat es sich bewährt, die *dimensionalen Spalten* (also die Primär & Fremdschlüssel der Tabelle, nach der Datensätze selektiert oder gruppiert werden) nach vorne zu stellen und die *kennzahlbezogenen Spalten* (die in den Auswertungen aggregiert werden) nach hinten zu stellen. In der Tabelle `F_FAELLE` stehen entsprechend Spalten zu Einrichtung, Aufnahme-/Entlasszeitpunkt und Hauptdiagnose vorne, während numerische Attribute wie das Aufnahmealter und die Liegedauer am Ende stehen.

In [ ]:
%%sql $db_url_staging
DELETE FROM reporting.f_faelle;

INSERT INTO reporting.f_faelle (fall_id, patient_id, einrichtung_id, aufnahme_datum, aufnahme_datumzeit, entlass_datum, entlass_datumzeit, fallart_id, aufnahmegrund_id,
                                hauptdiagnose_snomed_id, aufnahmealter_jahre, liegedauer_tage, erloes_fallpauschale)

SELECT 'B-' || fal.fall_id                 AS fall_id,
       'B-' || fal.patient_id              AS patient_id,
       'B-' || fal.einrichtung_id          AS einrichtung_id,
       SUBSTR(fal.aufnahmedatum, 1, 10)    AS aufnahme_datum,
       fal.aufnahmedatum                   AS aufnahme_datumzeit,
       SUBSTR(fal.entlassdatum, 1, 10)     AS entlass_datum,
       fal.entlassdatum                    AS entlass_datumzeit,
       frt.fallart_id,
       fal.aufnahmegrund                   AS aufnahmegrund_id,
       fal.hauptdiagnose                   AS hauptdiagnose_snomed_id,
       (JULIANDAY(fal.aufnahmedatum) - JULIANDAY(pat.geburtsdatum))/365 AS aufnahmealter_jahre,
       JULIANDAY(fal.entlassdatum)   - JULIANDAY(fal.aufnahmedatum)     AS liegedauer_tage,
       fal.erloes_fallpauschale
  FROM bielefeld.faelle    fal
  JOIN bielefeld.patienten pat ON fal.patient_id = pat.patient_id
  JOIN reporting.d_fallart frt ON fal.fallart    = frt.fallart_name

UNION

SELECT 'M-' || enc.id                      AS fall_id,
       'M-' || enc.patient_id              AS patient_id,
       'M-' || enc.organization_id         AS einrichtung_id,
       SUBSTR(enc.start, 1, 10)            AS aufnahme_datum,
       enc.start                           AS aufnahme_datumzeit,
       SUBSTR(enc.stop, 1, 10)             AS entlass_datum,
       enc.stop                            AS entlass_datumzeit,
       frt.fallart_id,
       enc.code                            AS aufnahmegrund_id,
       enc.reasoncode                      AS hauptdiagnose_snomed_id,
       (JULIANDAY(enc.start) - JULIANDAY(pat.birthdate))/365 AS aufnahmealter_jahre,
       JULIANDAY(stop)       - JULIANDAY(start)              AS liegedauer_tage,
       enc.base_encounter_cost             AS erloes_fallpauschale
  FROM mannheim.encounters enc
  JOIN mannheim.patients   pat ON enc.patient_id     = pat.id
  JOIN reporting.d_fallart frt ON enc.encounterclass = frt.fallart_name

### 3. Indizes ergänzen

Da Faktentabellen in der Regel umfangreich und über die Zeit wachsend sind, sollten Indizes auf Spalten angelegt werden, die als Selektionskriterien für die erwarteten Auswertungen genutzt werden. Die Datenbank kann die Indizes für die schnelle Selektion relevanter Datensätze nutzen und vermeidet so zeitaufwändige *table scans*. Im DWH-Bereich werden Indizes typischerweise auf Spalten angelegt, die über Fremdschlüssel mit Dimensionstabellen verbunden sind. Darüber hinaus sollten Spalten indiziert werden, die als Primär- oder Fremdschlüssel für die Verknüpfung mit anderen Faktentabellen verwendet werden. Primärschlüssel sollten mit einem `UNIQUE` Index angelegt werden, da die Datenbank dann die Anlage doppelter Einträge verhindert und nach dem ersten Treffer nicht weitersuchen muss.





In [ ]:
%%sql $db_url_reporting
CREATE UNIQUE INDEX IF NOT EXISTS ix_fal_fall_id ON f_faelle (fall_id);
CREATE INDEX IF NOT EXISTS ix_fal_patient_id ON f_faelle (patient_id);
CREATE INDEX IF NOT EXISTS ix_fal_aufnahme_datum ON f_faelle (aufnahme_datum);
CREATE INDEX IF NOT EXISTS ix_fal_entlass_datum ON f_faelle (entlass_datum);
CREATE INDEX IF NOT EXISTS ix_fal_fallart_id ON f_faelle (fallart_id);
CREATE INDEX IF NOT EXISTS ix_fal_aufnahmegrund_id ON f_faelle (aufnahmegrund_id);
CREATE INDEX IF NOT EXISTS ix_fal_hauptdiagnose_snomed_id ON f_faelle (hauptdiagnose_snomed_id);

### 4. Ergebniskontrolle

Anschließend können wir mit einer `SELECT`-Abfrage überprüfen, ob die Tabelle korrekt erstellt wurde.


In [ ]:
%%sql $db_url_staging
SELECT *
  FROM reporting.f_faelle
 LIMIT 10

## Faktentabelle F_DIAGNOSEN

Im Folgenden erstellen wir eine weitere Faktentabelle, bei der wir Spalten aus der vorher generierten F_FAELLE übernehmen und um Angaben zu den Diagnosen der Patient:innen ergänzen.

Grundsätzlich könnte die Tabelle F_FAELLE auch zur Laufzeit einer Abfrage an eine schmalere Tabelle ausschließlich mit Diagnosedaten gejoined werden, aber der Star-Schema-Ansatz zielt gerade darauf ab, in einem in sich geschlossenen Star-Schema zu einem Thema zentral nur eine Faktentabelle mit den zugehörigen Dimensionstabellen zu haben.

### 1. Zieltabelle anlegen

Hierzu legen wir zunächst mit einem CREATE-Table die Tabelle in der Reporting-Datenbank (leer) an.

In [ ]:
%%sql $db_url_staging
DROP TABLE IF EXISTS reporting.f_diagnosen;

CREATE TABLE reporting.f_diagnosen (
  fall_id                 INTEGER,
  patient_id              INTEGER,
  einrichtung_id          INTEGER,
  aufnahme_datum          VARCHAR(10),
  aufnahme_datumzeit TEXT VARCHAR(18),
  entlass_datum           VARCHAR(10),
  entlass_datumzeit TEXT  VARCHAR(18),
  fallart_id              INTEGER,
  aufnahmegrund_id        VARCHAR(20),
  hauptdiagnose_snomed_id VARCHAR(20),
  diagnose_datum          VARCHAR(10),
  diagnose_snomed_id      VARCHAR(20),
  aufnahmealter_jahre     FLOAT,
  liegedauer_tage         FLOAT,
  erloes_fallpauschale    FLOAT
);

### 2. Daten aus der Quelltabelle auslesen & in Zieltabelle schreiben

Die fallbezogenen Daten können aus der Tabelle F_FAELLE übernommen werden, so dass die Spalten nicht redundant wieder aus den Quelldaten erzeugt werden müssen (Caveat: Reihenfolge der ETL-Jobs beachten).
Da in der Tabelle F_FAELLE die Fallnummern der Standorte über ein Präfix qualifiziert wurden, muss dies auch beim JOIN mit den Diagnose-Quelldaten der Standorte einbezogen werden.

In [ ]:
%%sql $db_url_staging
DELETE FROM reporting.f_diagnosen;

INSERT INTO reporting.f_diagnosen (fall_id, patient_id, einrichtung_id, aufnahme_datum, aufnahme_datumzeit, entlass_datum, entlass_datumzeit, fallart_id, aufnahmegrund_id,
                                   hauptdiagnose_snomed_id, diagnose_datum, diagnose_snomed_id, aufnahmealter_jahre, liegedauer_tage, erloes_fallpauschale)

SELECT fal.fall_id,
       fal.patient_id,
       fal.einrichtung_id,
       fal.aufnahme_datum,
       fal.aufnahme_datumzeit,
       fal.entlass_datum,
       fal.entlass_datumzeit,
       fal.fallart_id,
       fal.aufnahmegrund_id,
       fal.hauptdiagnose_snomed_id,
       dia.diagnose_datum,
       dia.diagnose_code           AS diagnose_snomed_id,
       fal.aufnahmealter_jahre,
       fal.liegedauer_tage,
       fal.erloes_fallpauschale
  FROM reporting.f_faelle fal
  JOIN bielefeld.diagnosen dia ON fal.fall_id = 'B-' || dia.fall_id

UNION

SELECT fal.fall_id,
       fal.patient_id,
       fal.einrichtung_id,
       fal.aufnahme_datum,
       fal.aufnahme_datumzeit,
       fal.entlass_datum,
       fal.entlass_datumzeit,
       fal.fallart_id,
       fal.aufnahmegrund_id,
       fal.hauptdiagnose_snomed_id,
       cnd.start                   AS diagnose_datum,
       cnd.code                    AS diagnose_snomed_id,
       fal.aufnahmealter_jahre,
       fal.liegedauer_tage,
       fal.erloes_fallpauschale
  FROM reporting.f_faelle  fal
  JOIN mannheim.conditions cnd ON fal.fall_id = 'M-' || cnd.encounter_id

### 3. Indizes ergänzen

Auch diese Faktentabelle sollte Indizes wie zuvor bei der Tabelle F_FAELLE erhalten.
Caveat: Je nach Datenbank gibt es ggf. nur einen Namensraum für alle Indizes eines Schemas, so dass die Indizes eindeutig benanntt werden müssen (hier durch Aufnahme des Kürzels `dia` in den Namen des jeweiligen Index).

In [ ]:
%%sql $db_url_reporting
CREATE INDEX IF NOT EXISTS ix_dia_fall_id                 ON f_diagnosen (fall_id);
CREATE INDEX IF NOT EXISTS ix_dia_patient_id              ON f_diagnosen (patient_id);
CREATE INDEX IF NOT EXISTS ix_dia_aufnahme_datum          ON f_diagnosen (aufnahme_datum);
CREATE INDEX IF NOT EXISTS ix_dia_entlass_datum           ON f_diagnosen (entlass_datum);
CREATE INDEX IF NOT EXISTS ix_dia_fallart_id              ON f_diagnosen (fallart_id);
CREATE INDEX IF NOT EXISTS ix_dia_aufnahmegrund_id        ON f_diagnosen (aufnahmegrund_id);
CREATE INDEX IF NOT EXISTS ix_dia_hauptdiagnose_snomed_id ON f_diagnosen (hauptdiagnose_snomed_id);
CREATE INDEX IF NOT EXISTS ix_dia_diagnose_snomed_id      ON f_diagnosen (diagnose_snomed_id);
CREATE INDEX IF NOT EXISTS ix_dia_diagnose_datum          ON f_diagnosen (diagnose_datum);

### 3. Ergebniskontrolle

Anschließend können wir mit einer `SELECT`-Abfrage überprüfen, ob die Tabelle korrekt erstellt wurde.

In [ ]:
%%sql $db_url_staging
SELECT *
  FROM reporting.f_diagnosen
 LIMIT 10